# Experiements with voting algorithms

## 1. Maximum Likelihood Voting vs Weighted Average vs Approximate Majority Voting on data with given confidence

We use a sample of 3 values, each with a given confidence level. This simulates the way output from a machine learning model could possibly be weighted. We vote to select the most likely value using three algorithms:
- Maximum likelihood voting
- Weighted average voting, using the confidence as weights
- Approximate Majority Voting, ignoring the confidence scores

The last 2 methods are implemented using VDX. We compare the selected value and execution time for 1 million iterations.

In [1]:
import vdx
import time

In [2]:
input = [0.9, 1.0, 1.1]
weights = [0.9, 0.9, 0.9]

In [3]:
def mlv(input, weights, margin):
    deltas = []
    for i in range(len(input)):
        delta = []
        for y in range(len(input)):
            if i != y:
                k = abs(input[i] - input[y])
                # round k to the nearest 0.1
                k = round(k * 100) / 100
                #print(k)
                if k <= margin:
                    delta.append(weights[i])
                else:
                    p = (1-weights[i])/(len(input) - 1)
                    p = round(p * 100) / 100
                    delta.append(p)
            #print(delta)
        # append the product of the deltas to the deltas list
        product = 1
        for item in delta:
            product *= item
            product = round(product * 100) / 100
        deltas.append(product)
    # find the index of the largest delta
    #print(deltas)
    index = deltas.index(max(deltas))
    # return the input at the index
    return input[index]

In [5]:
result1 = []
start = time.time() 
for i in range(1000000):
    p = mlv(input, weights, 0.1)
    result1.append(p)
end = time.time()
print("done in " + str(end - start) + " seconds")

done in 4.032844066619873 seconds


In [6]:
result2 = []
start = time.time()
for i in range(1000000):
    p = vdx.weighted_average(input, weights)
    p = vdx.nearest_neighbor(p, input)
    result2.append(p)
end = time.time()
print("done in " + str(end - start) + " seconds")

done in 1.6403329372406006 seconds


In [7]:
result3 = []
start = time.time()
for i in range(1000000):
    p = vdx.majority_voting_bootstrapping(input, 0.1)
    p = vdx.nearest_neighbor(p, input)
    result3.append(p)
end = time.time()
print("done in " + str(end - start) + " seconds")

done in 3.045283317565918 seconds


One interesting observation is that MLV prioritises the agreements between values, whereas the weighted average voting prioritises the weights. In other words, the value with the highest total weight would win the vote for the weighted vote, while a majority agreement would win the MLV vote, even if the sum of confidence scores is less than that of a smaller group of values.

## 2. Meanshift clustering vs Approximate Majority Voting vs X-Means clustering on multidimensional data

We use multidimensional samples to compare 3 algorithms for selecting the optimal data point.
- Meanshift clustering, where we select the value closest to the centroid of the biggest cluster
- Approximate Majority Voting on each dimension separately. We essentially compute the center of mass of the sample and select the closest point.
- X-Means clustering, where we select the value closest to the centroid of the biggest cluster

In [8]:
from sklearn.cluster import MeanShift
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
import numpy as np
import vdx
import time
import math

X = np.array([[1, 1, 2, 3,1, 1, 2, 3], [2, 1, 4, 2,1, 1, 2, 3], [1, 0, 1, 1,1, 1, 2, 3],
              [4, 7, 1, 3,1, 1, 2, 3], [3, 5, 2, 2,1, 1, 2, 3], [1, 1, 5, 6,1, 1, 2, 3],
               [2, 1, 0, 6,1, 1, 2, 3], [1, 0, 4, 4,1, 1, 2, 3],
              [4, 7, 1, 1,1, 1, 2, 3], [3, 5, 2, 2,1, 1, 2, 3]])

In [9]:
start = time.time()
for t in range(1000):
    clustering = MeanShift(bandwidth=2).fit(X)
    #print(clustering.labels_)
    # find the number of points in each cluster
    counts = np.bincount(clustering.labels_)
    #print(counts)
    # find the index of the max element in the counts array
    index = np.argmax(counts)
    #print(index)
    # find the center of the cluster with the max index
    center = clustering.cluster_centers_[index]
    #print(center)
    # find the point in X that is closest to the center
    closest = np.argmin(np.linalg.norm(X - center, axis=1))
print(X[closest])
end = time.time()
print("done in " + str(end - start) + " seconds")

[4 7 1 3 1 1 2 3]
done in 7.204146146774292 seconds


In [10]:
start = time.time()
for t in range(1000):
    output = []
    for i in range(len(X[0])):
        input = []
        for j in range(len(X)):
            input.append(X[j][i])
        #print("input: " + str(input))
        error = 0.1
        vote = vdx.majority_voting_bootstrapping(input, error)
        vote = vdx.nearest_neighbor(vote, input)
        output.append(vote)
# find the point in X that is closest to the center
closest = np.argmin(np.linalg.norm(X - output, axis=1))
print(X[closest])
end = time.time()
print("done in " + str(end - start) + " seconds")

[1 1 2 3 1 1 2 3]
done in 0.17012286186218262 seconds


In [11]:
sample = np.array([[1, 1, 2, 3,1, 1, 2, 3], [2, 1, 4, 2,1, 1, 2, 3], [1, 0, 1, 1,1, 1, 2, 3],
              [4, 7, 1, 3,1, 1, 2, 3], [3, 5, 2, 2,1, 1, 2, 3], [1, 1, 5, 6,1, 1, 2, 3],
               [2, 1, 0, 6,1, 1, 2, 3], [1, 0, 4, 4,1, 1, 2, 3],
              [4, 7, 1, 1,1, 1, 2, 3], [3, 5, 2, 2,1, 1, 2, 3]])
#print(sample)
# Prepare initial centers - amount of initial centers defines amount of clusters from which X-Means will
# start analysis.
start = time.time()
for t in range(1000):
    amount_initial_centers = 2
    initial_centers = kmeans_plusplus_initializer(sample, amount_initial_centers).initialize()
    # Create instance of X-Means algorithm. The algorithm will start analysis from 2 clusters, the maximum
    # number of clusters that can be allocated is 20.
    xmeans_instance = xmeans(sample, initial_centers, 20)
    xmeans_instance.process()
    # Extract clustering results: clusters and their centers
    clusters = xmeans_instance.get_clusters()
    #print(clusters)
    centers = xmeans_instance.get_centers()
    #print(centers)
    # Get the index of the cluster with the most points
    
    max_length = 0
    max_index = 0
    for i in range(len(clusters)):
        if len(clusters[i]) > max_length:
            max_length = len(clusters[i])
            max_index = i
    # Get the center of the cluster with the most points
    center = centers[max_index]
    #print(center)
    min_distance = math.inf
    # Get the index of the point in the sample that is closest to the center
    for i in range(len(clusters[max_index])):
        distance = math.dist(sample[clusters[max_index][i]], center)
        if distance < min_distance:
            min_distance = distance
            min_index = clusters[max_index][i]
print(sample[min_index])

end = time.time()
print("done in " + str(end - start) + " seconds")

[1 1 2 3 1 1 2 3]
done in 0.7453997135162354 seconds


We observe a great difference in performance between the algorithms. Interestingly, meanshift disagrees with the other 2 on the output.